In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
import torch
name = "bigcode/starcoder2-15b"
model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained(name)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:03<00:00,  3.99it/s]


In [13]:
import datasets
lang = "racket"
multiplt = datasets.load_dataset("nuprl/MultiPL-T", split=lang)

In [18]:
# just a random func
import random
fn = random.choice(multiplt["content"])
print(fn)
entrypoint = fn.split("(define (")[1].split(" ")[0]

;; Return the length of a base directory path, including the last '/'.
(define (base_dir_length base_dir_path)
	(if (equal? (string-ref base_dir_path (- (string-length base_dir_path) 1)) #\/)
		(string-length base_dir_path)
		(+ (string-length base_dir_path) 1)))


In [ ]:
prompt = f"""<issue_start>username_0: You are exceptionally skilled at crafting high-quality programming problems and
offering precise solutions.
Please gain inspiration from the following random code snippet to create a
high-quality programming problem in {lang.title()}. Present your output in two distinct sections:
[Problem Description] and [Solution].

Code snippet for inspiration:
```{lang}
{fn}
```
Guidelines for each section:
1. [Problem Description]: This should be **completely self-contained**, providing
all the contextual information one needs to understand and solve the problem.
Assume common programming knowledge, but ensure that any specific context,
variables, or code snippets pertinent to this problem are explicitly included.
2. [Solution]: Offer a comprehensive, **correct** solution that accurately
addresses the [Problem Description] you provided.<issue_comment>username_1: Sure, no problem. I will be able to help. I am  exceptionally skilled at crafting high-quality programming problems and
offering precise solutions.
I will use your provided code snippet for {entrypoint} as inspiration for the problem.

# [Problem Description]
"""

toks = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
gen = model.generate(toks, max_new_tokens=700, do_sample=True, temperature=0.4, top_p=0.9)[0]
dec = tokenizer.decode(gen[len(toks[0]):])
print(dec)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
